# TOC:
* 1.[Extract Movie list](#No1)
* 2.[Extract Movie Detail](#No2)
* 3.[Extract Movie awards](#No3)
* 4.[Extract Movie keywords](#No4)
* 5.[Extract Movie Rate](#No5)
* 6.[Merge data](#No6)
* 7.[Process awards](#No7)
* 8.[Merge detail and awards](#No8)
* 9.[Buildup model](#No9)

In [4]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from requests_futures.sessions import FuturesSession
from ediblepickle import checkpoint
from collections import namedtuple
import os
import locale
import sys
import  spacy
import time
import json
import pandas as pd
import collections
from retrying import retry
import re
import glob,os,sys
import simplejson,json
import spacy
nlp = spacy.load("en_core_web_sm")
from functools import reduce
sys.setrecursionlimit(1000000)

URL_template = '''https://www.imdb.com/search/title/?title_type=feature&year={YEAR}-01-01,{YEAR}-12-31&start={START}&ref_=adv_nxt'''

#'''https://www.imdb.com/search/title/?title_type=feature&year={YEAR}-01-01,{YEAR}-12-31&start={START}&ref_=adv_nxt'''



# Extract Movie list #1 <a class="anchor" id="No1"></a>
### the top 2000 populest movie of each year from 1990 - 2019

In [188]:
@retry(wait_fixed=12000,stop_max_delay=10000)
def GetListFromLink(url,year,start):
    url = url.format(YEAR=year,START=start)
    result = namedtuple('info',['title','year','popularity_rank_year',
                                'imdb_rate','imdb_id','RunTime',
                                'certification','genre','MetaScore',
                               'votes','gross'])
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    parts = soup.select('div#main div.lister-list div.lister-item')
    res = []
    for part in parts:
        sub_part = part.find('div',class_='lister-item-content')
        popularity = int(sub_part.find(class_='lister-item-index').text[:-1].replace(',',''))

        title_part = sub_part.find('h3',class_='lister-item-header')
        title= title_part.find('a').text.strip()
        ID = title_part.find('a')['href'].split('/')[-2]
        year = title_part.find('span',class_='lister-item-year').text
        
        mute_part = sub_part.find('p',class_='text-muted')
        if mute_part.find('span',class_='runtime'):
            RunTime =mute_part.find('span',class_='runtime').text.split()[0]
        else:
            RunTime = None
        if mute_part.find('span',class_='certificate'):
            cert = mute_part.find('span',class_='certificate').text
        else:
            cert = None
        if mute_part.find('span',class_='genre'):
            genre = mute_part.find('span',class_='genre').text.strip()
        else:
            genre = None
            
        rate_part = sub_part.find('div',class_='ratings-bar')
        if rate_part:
            rate = rate_part.find('div',class_='ratings-imdb-rating')
            if rate:
                rate = rate_part.find('div',class_='ratings-imdb-rating').find('strong').text.strip()
            else:
                rate= None

            MataScore = rate_part.find('div',class_='ratings-metascore')
            if MataScore:
                MataScore = MataScore.find('span',class_='metascore').text.strip()
            else:
                MataScore = None
        else:
            rate = None
            MataScore = None
        
        score_part = sub_part.find('p',class_='sort-num_votes-visible')
        if score_part:
            Votes = score_part.find_all('span',{'name':'nv'})
            if Votes :
                Votes = score_part.find_all('span',{'name':'nv'})[0]['data-value']
            else:
                Votes= None
            if len(score_part.find_all('span',{'name':'nv'}))>1:
                Gross = score_part.find_all('span',{'name':'nv'})[1]['data-value']
            else:
                Gross = None
        else:
            Votes= None
            Gross = None
#        stars = 
#        director = 

        IMDBrate =rate
        res.append(result(title,year,popularity,IMDBrate,ID,RunTime,cert,
                         genre,MataScore,Votes,Gross))
    return res

In [191]:
#df = pd.DataFrame(columns=['name','year','popularity','rate','link'])
for year in range(1990,2020):
    df = pd.DataFrame(columns=['name', 'year', 'popularity_rank_year', 'imdb_rate', 'imdb_link'])
    for start in range(1,2001,50):
        sys.stdout.write('process {}:{}/40'.format(year,int((start-1)/50)+1)+'\n')
        sys.stdout.flush()
        tmp = GetListFromLink(URL_template,year,start)
        for element in tmp:
            new_row = pd.Series(data={x:y for x,y in element._asdict().items()})
            df = df.append(new_row,ignore_index=True)
    df.to_csv('cache/{}.csv'.format(year))
    
    if year%5 ==4:
        time.sleep(120)

process 1990:1/40
process 1990:2/40
process 1990:3/40
process 1990:4/40
process 1990:5/40
process 1990:6/40
process 1990:7/40
process 1990:8/40
process 1990:9/40
process 1990:10/40
process 1990:11/40
process 1990:12/40
process 1990:13/40
process 1990:14/40
process 1990:15/40
process 1990:16/40
process 1990:17/40
process 1990:18/40
process 1990:19/40
process 1990:20/40
process 1990:21/40
process 1990:22/40
process 1990:23/40
process 1990:24/40
process 1990:25/40
process 1990:26/40
process 1990:27/40
process 1990:28/40
process 1990:29/40
process 1990:30/40
process 1990:31/40
process 1990:32/40
process 1990:33/40
process 1990:34/40
process 1990:35/40
process 1990:36/40
process 1990:37/40
process 1990:38/40
process 1990:39/40
process 1990:40/40
process 1991:1/40
process 1991:2/40
process 1991:3/40
process 1991:4/40
process 1991:5/40
process 1991:6/40
process 1991:7/40
process 1991:8/40
process 1991:9/40
process 1991:10/40
process 1991:11/40
process 1991:12/40
process 1991:13/40
process 199

process 2000:38/40
process 2000:39/40
process 2000:40/40
process 2001:1/40
process 2001:2/40
process 2001:3/40
process 2001:4/40
process 2001:5/40
process 2001:6/40
process 2001:7/40
process 2001:8/40
process 2001:9/40
process 2001:10/40
process 2001:11/40
process 2001:12/40
process 2001:13/40
process 2001:14/40
process 2001:15/40
process 2001:16/40
process 2001:17/40
process 2001:18/40
process 2001:19/40
process 2001:20/40
process 2001:21/40
process 2001:22/40
process 2001:23/40
process 2001:24/40
process 2001:25/40
process 2001:26/40
process 2001:27/40
process 2001:28/40
process 2001:29/40
process 2001:30/40
process 2001:31/40
process 2001:32/40
process 2001:33/40
process 2001:34/40
process 2001:35/40
process 2001:36/40
process 2001:37/40
process 2001:38/40
process 2001:39/40
process 2001:40/40
process 2002:1/40
process 2002:2/40
process 2002:3/40
process 2002:4/40
process 2002:5/40
process 2002:6/40
process 2002:7/40
process 2002:8/40
process 2002:9/40
process 2002:10/40
process 200

process 2011:35/40
process 2011:36/40
process 2011:37/40
process 2011:38/40
process 2011:39/40
process 2011:40/40
process 2012:1/40
process 2012:2/40
process 2012:3/40
process 2012:4/40
process 2012:5/40
process 2012:6/40
process 2012:7/40
process 2012:8/40
process 2012:9/40
process 2012:10/40
process 2012:11/40
process 2012:12/40
process 2012:13/40
process 2012:14/40
process 2012:15/40
process 2012:16/40
process 2012:17/40
process 2012:18/40
process 2012:19/40
process 2012:20/40
process 2012:21/40
process 2012:22/40
process 2012:23/40
process 2012:24/40
process 2012:25/40
process 2012:26/40
process 2012:27/40
process 2012:28/40
process 2012:29/40
process 2012:30/40
process 2012:31/40
process 2012:32/40
process 2012:33/40
process 2012:34/40
process 2012:35/40
process 2012:36/40
process 2012:37/40
process 2012:38/40
process 2012:39/40
process 2012:40/40
process 2013:1/40
process 2013:2/40
process 2013:3/40
process 2013:4/40
process 2013:5/40
process 2013:6/40
process 2013:7/40
process 2

# Extract Movie Detail #2 <a class="anchor" id="No2"></a>
### based on the movie list in #1
### from ExtractRapidAPI Movie Database 
### https://rapidapi.com/rapidapi/api/movie-database-imdb-alternative?endpoint=apiendpoint_843d3708-42a9-4240-8a68-2ced0372c20f

In [54]:
@retry(stop_max_attempt_number=10,stop_max_delay=100000,wait_fixed=2000)
def ExtractRapidAPI_Movie_Database(IMDbID):
    global received_data
    url = "https://movie-database-imdb-alternative.p.rapidapi.com/"

    querystring = {"i":'tt'+IMDbID,"r":"json","plot":"short"}

    headers = {
        'x-rapidapi-host': "movie-database-imdb-alternative.p.rapidapi.com",
        'x-rapidapi-key': "ba3fac27ecmsh223a6aba79ff747p1e6055jsn08473fcd1a00"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    return response #json.loads(response.text.replace('\\s','\\ s'))

In [56]:
tmp_df = pd.DataFrame(pd.read_csv('data/links.csv',dtype={'movieId':str,'imdbId':str,'tmdbId':str}))
total = tmp_df['imdbId'].shape[0]
Index = np.linspace(0,total,31,dtype=int)
for part,ii in enumerate([(Index[i],Index[i+1]) for i in range(30)]):
    start = ii[0]
    end = ii[1]
    number_part = end-start
    df = pd.DataFrame()
    for index,ID in enumerate(tmp_df['imdbId'][start:end]):
        if index%100 ==0:
            print('processing : Part{}-{}%'.format(part,100*float(index)/float(number_part)))
        receive = ExtractRapidAPI_Movie_Database(ID)
        times = 0
        failed = False
        while receive.status_code !=200:
            if times > 10:
                break
                failed = True
            time.sleep(2)
            receive = ExtractRapidAPI_Movie_Database(ID)
            times+=1

        if failed:
            continue

        receive = json.loads(receive.text.replace('\\s','\\ s'))
        new_row = pd.Series(data={x:y for x,y in receive.items()})
        df = df.append(new_row,ignore_index=True)
    df.to_csv('data/detail/MovieDetails_{}.csv'.format(part))

processing : Part0-0.0%
processing : Part0-5.1652892561983474%
processing : Part0-10.330578512396695%
processing : Part0-15.49586776859504%
processing : Part0-20.66115702479339%
processing : Part0-25.826446280991735%
processing : Part0-30.99173553719008%
processing : Part0-36.15702479338843%
processing : Part0-41.32231404958678%
processing : Part0-46.48760330578512%
processing : Part0-51.65289256198347%
processing : Part0-56.81818181818182%
processing : Part0-61.98347107438016%
processing : Part0-67.14876033057851%
processing : Part0-72.31404958677686%
processing : Part0-77.47933884297521%
processing : Part0-82.64462809917356%
processing : Part0-87.8099173553719%
processing : Part0-92.97520661157024%
processing : Part0-98.14049586776859%
processing : Part1-0.0%
processing : Part1-5.162622612287042%
processing : Part1-10.325245224574084%
processing : Part1-15.487867836861126%
processing : Part1-20.65049044914817%
processing : Part1-25.81311306143521%
processing : Part1-30.97573567372225

processing : Part10-98.14049586776859%
processing : Part11-0.0%
processing : Part11-5.162622612287042%
processing : Part11-10.325245224574084%
processing : Part11-15.487867836861126%
processing : Part11-20.65049044914817%
processing : Part11-25.81311306143521%
processing : Part11-30.97573567372225%
processing : Part11-36.13835828600929%
processing : Part11-41.30098089829634%
processing : Part11-46.463603510583376%
processing : Part11-51.62622612287042%
processing : Part11-56.78884873515746%
processing : Part11-61.9514713474445%
processing : Part11-67.11409395973155%
processing : Part11-72.27671657201859%
processing : Part11-77.43933918430562%
processing : Part11-82.60196179659268%
processing : Part11-87.76458440887971%
processing : Part11-92.92720702116675%
processing : Part11-98.08982963345379%
processing : Part12-0.0%
processing : Part12-5.1652892561983474%
processing : Part12-10.330578512396695%
processing : Part12-15.49586776859504%
processing : Part12-20.66115702479339%
processing

processing : Part21-67.11409395973155%
processing : Part21-72.27671657201859%
processing : Part21-77.43933918430562%
processing : Part21-82.60196179659268%
processing : Part21-87.76458440887971%
processing : Part21-92.92720702116675%
processing : Part21-98.08982963345379%
processing : Part22-0.0%
processing : Part22-5.1652892561983474%
processing : Part22-10.330578512396695%
processing : Part22-15.49586776859504%
processing : Part22-20.66115702479339%
processing : Part22-25.826446280991735%
processing : Part22-30.99173553719008%
processing : Part22-36.15702479338843%
processing : Part22-41.32231404958678%
processing : Part22-46.48760330578512%
processing : Part22-51.65289256198347%
processing : Part22-56.81818181818182%
processing : Part22-61.98347107438016%
processing : Part22-67.14876033057851%
processing : Part22-72.31404958677686%
processing : Part22-77.47933884297521%
processing : Part22-82.64462809917356%
processing : Part22-87.8099173553719%
processing : Part22-92.97520661157024

# Extract Movie awards #3 <a class="anchor" id="No3"></a>
## extract movie awards infomation and total rates
### https://rapidapi.com/apidojo/api/imdb8?endpoint=apiendpoint_6fe36a3a-85b0-4ebd-bea8-7fbc7fd5bb80

In [ ]:
@retry(stop_max_attempt_number=10,stop_max_delay=100000,wait_fixed=2000)
def GetAwards(imdbid):
    url = "https://imdb8.p.rapidapi.com/title/get-awards-summary"

    querystring = {"tconst":'tt'+str(imdbid)}

    headers = {
        'x-rapidapi-host': "imdb8.p.rapidapi.com",
        'x-rapidapi-key': "6b5fc29badmsha838e41667c3d21p1035f2jsnbd88f7017928"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    return response
def ProcessAwards(response,imdb_id):
    j = response.json()
    if not j:
        return False
    result = {}
    if 'awardsSummary' in j.keys():
        result['awardsSummary'] = j['awardsSummary']
    else:
        result['awardsSummary'] = []
    result['imdb'] = 'tt'+str(imdb_id)
        
    return result

In [ ]:
tmp_df = pd.DataFrame(pd.read_csv('data/links.csv',dtype={'movieId':str,'imdbId':str,'tmdbId':str}))
total = tmp_df['imdbId'].shape[0]
Index = np.linspace(0,total,31,dtype=int)
for part,ii in enumerate([(Index[i],Index[i+1]) for i in range(30)]):
    start = ii[0]
    end = ii[1]
    number_part = end-start
    df = pd.DataFrame()
    for index,ID in enumerate(tmp_df['imdbId'][start:end]):
        if index%100 ==0:
            print('processing : Part{}-{}%'.format(part,100*float(index)/float(number_part)))
        response = GetAwards(ID)
        times = 0
        failed = False
        print(ID)
        while response.status_code !=200:
            if times > 10:
                failed = True
                break
            time.sleep(2)
            response = GetAwards(ID)
            times+=1

        if failed:
            continue

        res= ProcessAwards(response,ID)
        if not res:
            continue
        new_row = pd.Series(data={x:y for x,y in res.items()})
        df = df.append(new_row,ignore_index=True)
    df.to_csv('data/awards/Movie_awards_{}.csv'.format(part))

# Extract Movie keywords #4 <a class="anchor" id="No4"></a>
## extract movie keywords and total rates
### https://developers.themoviedb.org/3/movies/get-movie-external-ids


In [305]:
def Get_Reco_simi_key_pic(ID):
    global backup
    recommand = '''https://api.themoviedb.org/3/movie/{}?api_key=70242b4b08c2a17f5b14f6aa52052aec&append_to_response=keywords,similar,recommendations'''.format(ID)   
    tmp = requests.get(recommand)
    return tmp

def Process_simi_key_pic(response):
    j = response.json()
    result = {}
    if 'poster_path' in j.keys():
        result['poster_path'] = j['poster_path']
    else:
        result['poster_path'] = None
   
    if 'imdb_id' in j.keys():
        result['imdb_id'] = j['imdb_id']
    else:
        result['imdb_id'] = None
    result['tmdb_id'] = j['id']    
    if 'similar' in j.keys():
        result['similar'] = [x['id'] for x in j['similar']['results']]
    else:
        result['similar'] = []
    if 'keywords' in j.keys(): 
        result['keywords'] = j['keywords']['keywords']
    else:
        result['keywords'] = []
    if 'recommendations' in j.keys():    
        result['recommendations'] = [x['id'] for x in j['recommendations']['results']]
    else:
        result['recommendations'] = []
    return result

In [ ]:
tmp_df = pd.DataFrame(pd.read_csv('data/links.csv',dtype={'movieId':str,'imdbId':str,'tmdbId':str}))
total = tmp_df['tmdbId'].shape[0]
Index = np.linspace(0,total,31,dtype=int)
for part,ii in enumerate([(Index[i],Index[i+1]) for i in range(30)]):
    start = ii[0]
    end = ii[1]
    number_part = end-start
    df = pd.DataFrame()
    for index,ID in enumerate(tmp_df['tmdbId'][start:end]):
        if index%100 ==0:
            print('processing : Part{}-{}%'.format(part,100*float(index)/float(number_part)))
        response = Get_Reco_simi_key_pic(ID)
        times = 0
        failed = False

        while response.status_code !=200:
            if times > 10:
                failed = True
                break
            time.sleep(2)
            response = Get_Reco_simi_key_pic(ID)
            times+=1

        if failed:
            continue
        res= Process_simi_key_pic(response)
        new_row = pd.Series(data={x:y for x,y in res.items()})
        df = df.append(new_row,ignore_index=True)
    df.to_csv('data/simi_key_pic/Movie_simi_key_pic_{}.csv'.format(part))

# Extract Movie Rate # 5 <a class="anchor" id="No5"></a>
## extract movie rate and its distribution
### https://rapidapi.com/apidojo/api/imdb8?endpoint=apiendpoint_6fe36a3a-85b0-4ebd-bea8-7fbc7fd5bb80


In [ ]:
@retry(stop_max_attempt_number=10,stop_max_delay=100000,wait_fixed=2000)
def GetRate(imdbid):
    url = "https://imdb8.p.rapidapi.com/title/get-ratings"

    querystring = {"tconst":'tt'+str(imdbid)}

    headers = {
        'x-rapidapi-host': "imdb8.p.rapidapi.com",
        'x-rapidapi-key': "f8c8010930msh37fb3205573aa88p113491jsn9a926db741f6"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    return response
def ProcessRate(response,imdb_id):
    j = response.json()
    if not j:
        return False
    result = {}
    if 'ratingCount' in j.keys():
        result['ratingCount'] = j['ratingCount']
    else:
        result['ratingCount'] = []
    if 'ratingsHistograms' in j.keys():
        result.update(j['ratingsHistograms'])
    else:
        j['ratingsHistograms'] = []
    result['imdb'] = 'tt'+str(imdb_id)
    return result

In [ ]:
tmp_df = pd.DataFrame(pd.read_csv('data/links.csv',dtype={'movieId':str,'imdbId':str,'tmdbId':str}))
df = pd.DataFrame()
t = 0
index = 0
total = tmp_df['imdbId'].shape[0]
Index = np.linspace(0,total,31,dtype=int)
for part,ii in enumerate([(Index[i],Index[i+1]) for i in range(30)]):
    start = ii[0]
    end = ii[1]
    number_part = end-start
    df = pd.DataFrame()
    for index,ID in enumerate(tmp_df['imdbId'][start:end]):
        if index%100 ==0:
            print('processing : Part{}-{}%'.format(part,100*float(index)/float(number_part)))
        response = GetRate(ID)
        times = 0
        failed = False
        print(ID)
        while response.status_code !=200:
            if times > 10:
                failed = True
                break
            time.sleep(2)
            response = GetRate(ID)
            times+=1

        if failed:
            continue

        res= ProcessRate(response,ID)
        if not res:
            continue
        new_row = pd.Series(data={x:y for x,y in res.items()})
        df = df.append(new_row,ignore_index=True)
    df.to_csv('data/rate/Movie_rate_{}.csv'.format(part))b

# Merge data #6  <a class="anchor" id="No6"></a>

In [441]:
conca = []
for dataset in glob.glob('data/detail/*csv'):
    conca.append(pd.read_csv(dataset))
result = pd.concat(conca)
result = result.dropna(subset=['Ratings'])
result['Runtime'] = result['Runtime'].apply(lambda x: int(x.replace(',','').split()[0]) if type(x)==str else 0)

def processYear(x):
    if '–' in str(x):
        years =[int(t) for t in  x.split('–') if t]
        return sum(years)//len(years)
    return int(x)
result['Year'] = result['Year'].apply(processYear)

def split_name(x):
    if type(x) in [int,float]:
        return None
    else:
        return x.split(',')
result['Actors'] = result['Actors'].apply(split_name)
result['Country'] = result['Country'].apply(split_name)
result['Genre'] = result['Genre'].apply(split_name)
result['Director'] = result['Director'].apply(split_name)
result['Language'] = result['Language'].apply(split_name)
result['Production'] = result['Production'].apply(split_name)
result['Rated'] = result['Rated'].astype(str)
result['Writer'] = result['Writer'].apply(split_name)

def process_Ratings(x):
    if type(x) in [int,float]:
        return None
    else:
        text = json.loads(x.replace("'","\""))
        result = {}
        for source in text:
            if source['Source'] == 'Rotten Tomatoes':
                result[source['Source']] = float(source['Value'].strip('%'))/100
            else:
                result[source['Source']] = eval(source['Value'])
        return result
result['Ratings'] = result['Ratings'].apply(process_Ratings)
result = result.drop(['Unnamed: 0'],axis=1)

result.to_csv('data/Movie_details.csv')

In [590]:
conca = []
for dataset in glob.glob('data/rate/*csv'):
    conca.append(pd.read_csv(dataset))
result = pd.concat(conca)
all_list = ['Aged 18-29', 'Aged 30-44', 'Aged 45+', 'Aged under 18',
       'Females', 'Females Aged 18-29', 'Females Aged 30-44',
       'Females Aged 45+', 'Females Aged under 18', 'IMDb Staff', 'IMDb Users',
       'Males', 'Males Aged 18-29', 'Males Aged 30-44', 'Males Aged 45+',
       'Males Aged under 18', 'Non-US users', 'Top 1000 voters', 'US users']
result[all_list] = result[all_list].applymap(lambda x : json.loads(x.replace("'","\"")) if type(x) not in [float,int] else None)
result = result.drop(['Unnamed: 0'],axis=1)
result.to_csv('data/Movie_rate.csv')


In [594]:
conca = []
for dataset in glob.glob('data/awards/*csv'):
    conca.append(pd.read_csv(dataset))
result = pd.concat(conca)
def process_awards(x):
    j = json.loads(x.replace("'","\"").replace('True','1').replace('False','0'))
    result = {}
    keys = j.keys()
 
    if 'otherNominationsCount' in keys:
        result['otherNominationsCount'] = j['otherNominationsCount']
    else:
        result['otherNominationsCount'] = None

    if 'otherWinsCount' in keys:
        result['otherWinsCount'] = j['otherWinsCount']
    else:
        result['otherWinsCount'] = None
        
    return  result
result['awardsSummary'] = result['awardsSummary'].apply(process_awards)
result['otherWinsCount'] = result['awardsSummary'].apply(lambda x : x.get('otherWinsCount'))
result['otherNominationsCount'] = result['awardsSummary'].apply(lambda x : x.get('otherNominationsCount'))
result = result.drop(['awardsSummary','Unnamed: 0'],axis=1)
result.to_csv('data/Movie_awards.csv')

In [714]:
conca = []
for dataset in glob.glob('data/simi_key_pic/*csv'):
    conca.append(pd.read_csv(dataset))
result = pd.concat(conca)
result = result.drop(['Unnamed: 0'],axis=1)
def process_keywords(x):
#    print(x)
    tmp  = re.sub(r'([0-9a-zA-Z])(\")([\s|\w]+)',r"\1'\3",x.replace('\'','"')).replace('\\xa0','').\
    replace(''' "n' ''',''' 'n' ''')
    res = json.loads(tmp)
    return [i['id'] for i in res]
result['keywords'] = result['keywords'].apply(process_keywords)
result.to_csv('data/Movie_keys.csv')

# Process Awards of oscar and golden global #7 <a class="anchor" id="No7"></a>

In [81]:
df_o = pd.read_csv('data/the_oscar_award.csv')
df_g = pd.read_csv('data/golden_globe_awards.csv')
df2 = pd.read_csv('data/Movie_details.csv')

In [82]:
df_o = df_o[['winner','category','year_ceremony','film','year_film']]
df_o['oscar_winned_awards'] = None
df_o.loc[df_o['winner']==True,'oscar_winned_awards'] = df_o['category']
df_o['oscar_nominee_year_awards'] = df_o['year_ceremony'].astype('str')+"_"+df_o['category']
df_o = df_o.rename(columns={"winner": "oscar_winned", "category": "oscar_nominee_awards", 
                            "year_ceremony": "oscar_nominee_year",'name':'film',})

In [83]:
function = {a:list for a in ['oscar_winned', 'oscar_nominee_awards', 'oscar_winned_awards','oscar_nominee_year_awards']}
function['year_film'] = lambda x : list(x)[0]
function['oscar_nominee_year'] = lambda x : list(x)[0]
df_o = df_o.groupby('film').agg(function)
df_o  = df_o.reset_index()

In [84]:
df_o.shape

(4833, 7)

In [85]:
df_o['oscar_winned'] = df_o['oscar_winned'].apply(np.any)
df_o['oscar_winned_awards'] = df_o['oscar_winned_awards'].apply(lambda x: [i for i in x if i])

In [86]:
df_o = df_o.sort_values(by=['year_film'])

In [87]:
df_g = df_g[~df_g['category'].str.contains('Television Series')]
df_g = df_g[['win','category','year_award','film','year_film']]
df_g['glob_winned_awards'] = None
df_g.loc[df_g['win']==True,'glob_winned_awards'] = df_g['category']
df_g['glob_nominee_year_awards'] = df_g['year_award'].astype('str')+"_"+df_g['category']
df_g = df_g.rename(columns={"win": "glob_winned", "category": "glob_nominee_awards", "year_award": "glob_nominee_year"})

In [88]:
function = {a:list for a in ['glob_winned', 'glob_nominee_awards', 'glob_winned_awards', 'glob_nominee_year_awards']}
function['year_film'] = lambda x : list(x)[0]
function['glob_nominee_year'] = lambda x : list(x)[0]
df_g = df_g.groupby('film').agg(function)
df_g  = df_g.reset_index()

In [89]:
df_g['glob_winned'] = df_g['glob_winned'].apply(np.any)
df_g['glob_winned_awards'] = df_g['glob_winned_awards'].apply(lambda x: [i for i in x if i])

In [90]:
df_g = df_g.sort_values(by=['year_film'])

In [91]:
def process_name_g(x):
    x = re.sub(r"\(\d{4}\)",'',x)
    x = re.sub(r"\(.*TV.*\)",'',x)
    x = re.sub(r"\(.*Vol.*\)",'',x)
    x = re.sub(r"\(.*Parts.*\)",'',x)
    x = re.sub(r"^'",'',x)
   # x = re.sub(r"^'",'',x)
    if ', The' in x:
         x = x.strip().split(',')[1]+' '+x.strip().split(',')[0]
    if re.search(r'\,\sA$',x) or re.search(r'\,\sAn$',x):
         x = x.split(',')[1].strip()+' '+x.split(',')[0].strip()
    if '$1,000,000 Duck' in x:
        x = 'The Million Dollar Duck'
    if '(500) Days of Summer' == x:
        x = '500 Days of Summer'
    if 'night, Mother' == x:
        x = 'Mother night'
    if 'Do Ankhen Barah Haath' in x:
        x = 'Two Eyes, Twelve Hands'
    if 'Star Is Born, A' ==x :
        x = 'A Star Is Born'
    if 'Dynasty' in x:
        x = 'Dynasty'
    if  x == 'Friends (Foreign Language Film)':
        x = 'Friends'
    if 'Interview with the Vampire' in x:
        x = 'The Vampire Chronicles'
    if 'Kiiroi karasu' in x:
        x = 'Yellow Crow'
    if "L'eau vive (Girl And The River)" == x:
        x = "L'eau vive"
    if x=="La ronde (Circle Of Love)":
        x = 'La ronde'
    if x == 'La vérité (The Truth)':
        x = 'La vérité'
    if x == 'Page Eight (Masterpiece)':
        x = 'Page Eight'
    if x == 'Sherlock: Season 2 (Masterpiece)':
        x = 'Masterpiece'
    if x == 'Tarahumara (Always Further On)':
        x= 'Tarahumara'
    if x == 'The Trials Of Oscar Wilde (The Man With The Green Carnation)':
        x = 'The Man With The Green Carnation'
    if x == 'Three Colours: Blue (Trois couleurs: Bleu)':
        x = 'Three Colours: Blue'
    if x == "Walt Disney's Wonderful World of Color (Davy Crockett)":
        x = "Walt Disney's Wonderful World of Color"
    if x == "Wir Wunderkinder (Aren't We Wonderful!)":
        x = 'Wir Wunderkinder'
    if x=='Ánimas Trujano (The Important Man)':
        x = 'Ánimas Trujano'
    if x=='The Bees And The Italians) Signore & signori (The Birds':
        x = "The Birds, the Bees and the Italians"
    if x=='Simple Plan, a':
        x = 'A Simple Plan'
    if ',' in x:
        text = x.split(',')[0]
        doc = nlp(text)
        for entity in doc.ents:
            if entity.label_ == 'GPE' or text =='Denmark':
                x = ''.join(x.split(',')[1:]).strip()
        if text=='Denmark':
            x = ''.join(x.split(',')[1:]).strip()
    return x.strip()

In [92]:
def process_name_o(x):
    x = re.sub(r"\(.*\)",'',x)
    x = x.replace(';','').replace(']','')
    return x.strip()

In [93]:
df_g['film'] = df_g['film'].apply(process_name_g)

In [94]:
df_o['film'] = df_o['film'].apply(process_name_o)

In [99]:
df_o = df_o.groupby('film').agg({ i : lambda t : reduce(lambda x,y: x+y, t ) for i in  ['oscar_winned', 'oscar_nominee_awards', 'oscar_winned_awards',
       'oscar_nominee_year_awards', 'year_film', 'oscar_nominee_year']})

In [100]:
df_g = df_g.groupby('film').agg({ i : lambda t : reduce(lambda x,y: x+y, t ) for i in  ['glob_winned', 'glob_nominee_awards', 'glob_winned_awards',
       'glob_nominee_year_awards', 'year_film', 'glob_nominee_year']})

In [101]:
df_o.to_csv('data/the_oscar_award_processed.csv')
df_g.to_csv('data/golden_globe_awards_processed.csv')

# Combine awards and details #8 <a class="anchor" id="No8"></a>

In [36]:
df_d = pd.read_csv('data/Movie_details.csv')
df_o = pd.read_csv('data/the_oscar_award_processed.csv')
df_o= df_o.rename(columns={'film':'oscar_film','year_film':'oscar_year_film'})
df_g = pd.read_csv('data/golden_globe_awards_processed.csv')
df_g= df_g.rename(columns={'film':'globe_film','year_film':'globe_year_film'})

In [37]:
df_d = df_d.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [38]:
df_o_tmp = pd.merge(df_d,df_o,how='left',left_on=['Title'],right_on=['oscar_film'])

In [39]:
df_o_tmp_1 = df_o_tmp[(df_o_tmp['Year'] <= df_o_tmp['oscar_year_film']+1)\
                      &  (df_o_tmp['Year'] >= df_o_tmp['oscar_year_film']-1)]

In [40]:
df_o_tmp_2 = df_o_tmp[df_o_tmp.oscar_year_film.isnull()]

In [41]:
df_o_final = pd.concat([df_o_tmp_1,df_o_tmp_2])

In [42]:
df_g_tmp = pd.merge(df_o_final,df_g,how='left',left_on=['Title'],right_on=['globe_film'])

In [43]:
df_g_tmp_1 = df_g_tmp[(df_g_tmp['Year'] <= df_g_tmp['globe_year_film']+1)\
                      &  (df_g_tmp['Year'] >= df_g_tmp['globe_year_film']-1)]

In [44]:
df_g_tmp_2 = df_g_tmp[df_g_tmp.globe_year_film.isnull()]

In [45]:
df_g_final = pd.concat([df_g_tmp_1,df_g_tmp_2])

In [46]:
def select_award(x):
    res = [0,0]
    if type(x)==float:
        return res
    ll = x.split('.')
    for ele in ll:
        if not 'Oscar' in ele:
            if re.search(r'[0-9]+ nominati',ele):
                res[0] = int(re.search(r'([0-9]+)\s(nominati)',ele).group(1))
            if re.search(r'[0-9]+ win',ele):
                #res[0] = re.search(r'[0-9]+',re.search(r'[0-9]+ win',ele))
                res[1] = int(re.search(r'([0-9]+)\s(win)',ele).group(1))
    return tuple(res)

In [47]:
df_g_final.loc[:,'Awards'] = \
df_g_final['Awards'].apply(select_award)
df_g_final['nomineed'] = df_g_final.Awards.apply(lambda x: x[0])
df_g_final['winned']= df_g_final.Awards.apply(lambda x: x[1] )
df_g_final = df_g_final.drop(['Awards'],axis=1)

NameError: name 're' is not defined

In [48]:
def _get_rt(x):
    dic = json.loads(x.replace('\'','"'))
    if 'Rotten Tomatoes' in dic.keys():
        return dic['Rotten Tomatoes']
    else:
        return None

In [49]:
df_g_final['Rotten Tomatoes Rate'] = \
df_g_final[df_g_final.Ratings.notnull()]['Ratings'].apply(_get_rt)

NameError: name 'json' is not defined

In [50]:
df_g_final = df_g_final.drop(['Ratings','Response','Plot','Website','Season','Episode','seriesID','totalSeasons',
                                  'oscar_film','Type','Error','globe_film','globe_year_film','oscar_year_film'
                                 ],axis=1)

In [51]:
df_g_final.to_csv('data/All_movie_detail.csv')

In [52]:
df_k = pd.read_csv('data/Movie_keys.csv')

In [53]:
df_k.loc[df_k.poster_path.notnull(),'poster_path'] = \
df_k[df_k.poster_path.notnull()].poster_path.apply(lambda x : 'https://image.tmdb.org/t/p/original' + x)

In [54]:
df_k = df_k.groupby('imdb_id').first()

In [55]:
df_k.to_csv('data/Movie_keys.csv')

In [56]:
df_final = pd.merge(df_k,df_g_final,how='right',left_on='imdb_id',right_on='imdbID')

In [57]:
def process_budget(x):
    x = x[1:]
    if 'k' in x:
        return locale.atof(x[:-2])*1000
    else:
        return locale.atof(x)
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')
df_final.loc[df_final.BoxOffice.notnull(),'BoxOffice'] = \
df_final.loc[df_final.BoxOffice.notnull(),'BoxOffice'].apply(process_budget)

NameError: name 'locale' is not defined

In [ ]:
df_final = df_final.drop(['DVD','Released'],axis=1)

In [ ]:
df_final.loc[df_final.Language.notnull(),'Language'] = \
df_final.loc[df_final.Language.notnull(),'Language'].apply(lambda x : None if x == ['None'] else x)

In [ ]:
df_final.loc[df_final.imdbVotes.notnull(),'imdbVotes'] = \
df_final.loc[df_final.imdbVotes.notnull(),'imdbVotes'].apply(locale.atoi)

In [58]:
def process_bool(x):
    if x=='True':
        return 1
    else:
        return 0   

In [59]:
df_final['oscar_winned'] = df_final['oscar_winned'].apply(process_bool)
df_final['glob_winned'] = df_final['glob_winned'].apply(process_bool)

In [61]:
tmp = df_final[['keywords','Actors','Director', 'Genre', 'Language','Production', 'Rated','Writer',
           'oscar_nominee_awards', 'oscar_winned_awards','oscar_nominee_year_awards', 'oscar_nominee_year',
           'glob_nominee_awards', 'glob_winned_awards', 'glob_nominee_year_awards','glob_nominee_year']]

In [165]:
df_final[['imdbID','poster_path']].to_csv('data/fastlink_poster.csv')
df_final[['imdbID','tmdb_id']].to_csv('data/fastlink.tmdbID.csv')
df_final[['imdbID','Title']].to_csv('data/movie_Name.csv')
#df_final[['imdbID','Title']].to_csv('data/movie_Name.csv')
df_final = df_final.drop(['Poster','poster_path','tmdb_id','Title'], axis=1)
df_final.to_csv('data/final.csv')

# Combine awards and details #9 <a class="anchor" id="No9"></a>

In [156]:
from sklearn import base
from sklearn.feature_extraction import DictVectorizer, FeatureHasher
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import RidgeCV, LinearRegression, SGDRegressor, Ridge
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast 

In [160]:
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.fillna('[]')
        X = X.apply(ast.literal_eval)
        def to_dict(l):
            if l ==[]:
                return {}
            else:
                return {i:1 for i in l}
        
        return X.apply(to_dict)

In [3]:
df_final = pd.read_csv('data/final.csv')

In [157]:
cat_col_1 = ['keywords','Actors','Director', 'Genre', 'Language','Production','Writer',
           'oscar_nominee_awards', 'oscar_winned_awards','oscar_nominee_year_awards', 
           'glob_nominee_awards', 'glob_winned_awards', 'glob_nominee_year_awards']
cat_col_2 = ['glob_winned','oscar_winned','oscar_nominee_year','glob_nominee_year']
cat_col_3 = ['Country','Rated']
num_col = ['budget','revenue','BoxOffice','Metascore','imdbRating','imdbVotes']
cat_pipe_1 = Pipeline([
    ('encoder', DictEncoder()),
    ('vectorizer', FeatureHasher())
])
cat_pipe_2 = Pipeline([
    ('simpleimputer1', SimpleImputer(strategy='constant',fill_value=0)),
    ('vectorizer1', OneHotEncoder())
])
cat_pipe_3 = Pipeline([
    ('simpleimputer2', SimpleImputer(strategy='constant',fill_value='Nane')),
    ('vectorizer2', OneHotEncoder())
])
num_pipe = Pipeline([
   ('simpleimputer3', SimpleImputer(strategy='constant',fill_value=0))
])

In [161]:
tmp = DictEncoder().fit_transform(df_final['Actors'])
FeatureHasher().fit_transform(tmp)

<56665x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 215343 stored elements in Compressed Sparse Row format>

In [81]:
full_pipe = ColumnTransformer(
    
    [('c2',cat_pipe_2,cat_col_2) ,  
    ('c3',cat_pipe_3,cat_col_3) ,
    ('n',num_pipe,num_col)]
)
train = full_pipe.fit_transform(df_final)

ValueError: malformed node or string: 0               [733, 154802]
1                    [263548]
2        [1318, 4552, 191108]
3                          []
4                          []
                 ...         
56660                      []
56661                      []
56662                      []
56663                      []
56664                      []
Name: keywords, Length: 56665, dtype: object

In [246]:
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names  # We will need these in transform()
    
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def predict(self, X):
        return X[col_names]
    def transform(self, X):
        return X[col_names]
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names

In [27]:
df_d = pd.read_csv('data/All_movie_detail.csv',low_memory=False)

In [7]:
df_d

Index(['Unnamed: 0', 'Actors', 'Awards', 'BoxOffice', 'Country', 'DVD',
       'Director', 'Genre', 'Language', 'Metascore', 'Poster', 'Production',
       'Rated', 'Released', 'Runtime', 'Title', 'Writer', 'Year', 'imdbID',
       'imdbRating', 'imdbVotes', 'oscar_winned', 'oscar_nominee_awards',
       'oscar_winned_awards', 'oscar_nominee_year_awards',
       'oscar_nominee_year', 'glob_winned', 'glob_nominee_awards',
       'glob_winned_awards', 'glob_nominee_year_awards', 'glob_nominee_year',
       'Rotten Tomatoes Rate'],
      dtype='object')

In [ ]:
category_est_step1 = Pipeline([
        ('col',ColumnSelectTransformer(['categories'])),
        ('DicEn',DictEncoder()),
        ('dicVec',DictVectorizer()),
        # KNeighborsRegressor
    ])